# Store patterns efficiently

## Data

In [1]:
# two witnesses, with repetition and transposition

# Original example, single leaf node
w1 = '''the red and the black cat'''
w2 = '''the black and the red cat'''

# Adjacent transposition
# w1 = '''the red striped cat'''
# w2 = '''the striped red cat'''

# Two leaf nodes
# w1 = '''cat red black'''
# w2 = '''cat black red'''

# Branches meet in the middle at koala and then split again, with two leaf nodes
# w1 = """cat red black koala brown gray"""
# w2 = """cat black red koala gray brown"""

# Two split and rejoin
# w1 = '''the gray koala'''
# w2 = '''the brown koala'''

# medium example
# w1 = '''WHEN we look to the individuals of the same variety or sub-variety of
# our older cultivated plants and animals, one of the first points which strikes us, is,
# that they generally differ much more from each other, than do the individuals of any one
# species or variety in a state of nature.'''
# w2 = '''WHEN we look to the individuals of the same variety or sub-variety of
# our older cultivated plants and animals, one of the first points which strikes us, is,
# that they generally differ more from each other than do the individuals of any one
# species or variety in a state of nature.'''

# Larger example
# w1 = '''WHEN we look to the individuals of the same variety or sub-variety of
# our older cultivated plants and animals, one of the first points which strikes us, is,
# that they generally differ much more from each other, than do the individuals of any one
# species or variety in a state of nature. When we reflect on the vast diversity of the
# plants and animals which have been cultivated, and which have varied during all ages
# under the most different climates and treatment, I think we are driven to conclude that
# this greater variability is simply due to our domestic productions having been raised
# under conditions of life not so uniform as, and somewhat different from, those to which
# the parent-species have been exposed under nature. There is, also, I think, some
# probability in the view propounded by Andrew Knight, that this variability may be partly
# connected with excess of food. It seems pretty clear that organic beings must be exposed
# during several generations to the new conditions of life to cause any appreciable amount
# of variation; and that when the organisation has once begun to vary, it generally
# continues to vary for many generations. No case is on record of a variable being ceasing
# to be variable under cultivation. Our oldest cultivated plants, such as wheat, still
# often yield new varieties: our oldest domesticated animals are still capable of rapid
# improvement or modification.'''
# w2 = '''WHEN we look to the individuals of the same variety or sub-variety of
# our older cultivated plants and animals, one of the first points which strikes us, is,
# that they generally differ more from each other than do the individuals of any one
# species or variety in a state of nature. When we reflect on the vast diversity of the
# plants and animals which have been cultivated, and which have varied during all ages
# under the most different climates and treatment, I think we are driven to conclude that
# this great variability is simply due to our domestic productions having been raised
# under conditions of life not so uniform as, and somewhat different from, those to which
# the parent-species have been exposed under nature. There is also, I think, some
# probability in the view propounded by Andrew Knight, that this variability may be partly
# connected with excess of food. It seems pretty clear that organic beings must be exposed
# during several generations to the new conditions of life to cause any appreciable amount
# of variation; and that when the organisation has once begun to vary, it generally
# continues to vary for many generations. No case is on record of a variable being ceasing
# to be variable under cultivation. Our oldest cultivated plants, such as wheat, still
# often yield new varieties: our oldest domesticated animals are still capable of rapid
# improvement or modification'''

## Work plan

1. Create token array (Python **list**)
1. Create suffix array
1. Create LCP (**longest common prefix**) array
1. Calculate LCP intervals
1. Create patterns

## Construct list of ngrams shared by witnesses

Find ngrams and positions in witnesses

### Tokenize witnesses

In [2]:
def tokenize_witnesses(w1_string, w2_string):
    '''Return list of witnesses, each represented by a list of tokens'''
    # TODO: handle punctuation, upper- vs lowercase
    w1_tokens = w1.split()
    w2_tokens = w2.split()
    witnesses = [w1_tokens, w2_tokens]
    return witnesses

In [3]:
witnesses = tokenize_witnesses(w1, w2)
print(witnesses) # take a look

[['the', 'red', 'and', 'the', 'black', 'cat'], ['the', 'black', 'and', 'the', 'red', 'cat']]


In [4]:
# Create token array
# All tokens from both witnesses in a single list, with a separator (" # ") between witnesses
token_array = []
token_array.extend(witnesses[0])
token_array.append(" # ")
token_array.extend(witnesses[1])
[(index, value) for index,value in enumerate(token_array)] # take a look

[(0, 'the'),
 (1, 'red'),
 (2, 'and'),
 (3, 'the'),
 (4, 'black'),
 (5, 'cat'),
 (6, ' # '),
 (7, 'the'),
 (8, 'black'),
 (9, 'and'),
 (10, 'the'),
 (11, 'red'),
 (12, 'cat')]

In [5]:
# determine suffixes of token array
# output with print() is diagnostic
# for index, token in enumerate(token_array):
#     suffix = token_array[index:]
#     print(suffix)

In [6]:
# suffix array is sorted alphabetically
# tuples of list of tokens (suffix) and position in original list of tokens
# TODO: less naive implementation
suffixes = []
for index, token in enumerate(token_array):
    suffix = token_array[index:]
    suffixes.append((suffix, index))
suffixes.sort() # sort in place
# suffixes # take a look

Notice that suffixes that start at position 2 and 9 both start with "and the", which tells us that:

1. There is a repeated suffix "and the"
1. "and" appears without "the"

Ergo, we don't need a unigram "and".

Similarly, other ngrams occur repeatedly: "the red" twice, "the" four times, etc. Occurrences of "and" are easy because they are only "and the", while "the" occurs in different contexts, e.g., twice in "the red", twice in "the black".

In [7]:
# suffix array is list of offsets of start positions of suffixes sorted alphabetically
# Suffix array is economical because it is equal to the sum of the lengths of witnesses plus the number of witnesses - 1 (for the separators)
suffix_array = []
for suffix, index in suffixes:
    suffix_array.append(index)
suffix_array[80:90] # take a look

[]

In [8]:
# compute LCP array
# sequential pairs of values in suffix array, which are two offsets in the sorted suffixes
# for i in range(0, len(suffix_array) - 1):
#     print (suffix_array[i:i+2])

In [9]:
# compute LCP array, which is a sequence of integers representing the number of tokens shared by consecutive alphabetically sorted suffixes
# sequential pairs of values in suffix array, which are two offsets in the sorted suffixes
# TODO: we now need the longest prefix
lcp_array = [0]
for i in range(0, len(suffix_array) - 1): # for each pair of suffixes, retrieve list of tokens starting at that position
    pair = suffix_array[i:i+2] # for each pair of suffixes
    suffix_1 = token_array[pair[0]:] # tokens starting at first position
    suffix_2 = token_array[pair[1]:] # tokens starting at second position
    # print(suffix_1, suffix_2) # diagnostic: verify that they're paired correctly
    lcp_value = next(filter(lambda t: t[1][0] != t[1][1], enumerate(zip(suffix_1, suffix_2))), min(len(suffix_1), len(suffix_2))) # pair the tokens up by position, return (number of matches, first non-match)
    lcp_array.append(lcp_value[0] if type(lcp_value) == tuple else lcp_value) # most are tuples, but some are just an integer
# lcp_array # take a look

In [10]:
# Use LCP array to calculate patterns (efficiently)
# Values in LCP array represent lengths of matching ngrams
#   e.g., the three values of 2 are "and the", "the black", "the red"
#   "the" is harder: unigram appears four times, plus "the black" and "the red"
#
# Of interest:
#   1. 0 means that whatever follows will have nothing in common with it
#   2. Repetition of same number (doesn't happen here) means same pattern
#   3. Consecutive non-zero values identify how much of the pattern they have in common, 
#      e.g., end goes from "the black" (2) to "the" (1) to "the red" (2)
#         Counts are always +1, so there must be two instances of "the red", two of "the black", 
#      and four of "the" (two unigrams and two embedded in "the red" and "the black")

In [11]:
# create Block dataclass
from dataclasses import dataclass
@dataclass(unsafe_hash=True)
class Block:
    token_count: int
    start_position: int # offset into suffix array (not in token array!)
    end_position: int # start and end position give number of occurrences
    all_start_positions: [] # compute after blocks have been completed
    # witness_count: int # number of witnesses in which pattern occurs, omitted temporarily because requires further computation
    frequency: int # number of times pattern occurs in whole witness set (may be more than once in a witness), end_position - start_position + 1
    how_created: int # debug

In [12]:
# create blocks from the lcp array
from collections import deque # faster append and pop than list
blocks = []
open_block_stack = deque()
print(list(enumerate(lcp_array)))
for offset, lcp in enumerate(lcp_array):
    # three situations: next one is same value, higher that last, or lower than last
    # if same value: same pattern
    # if higher or lower, new pattern (may overlap with previous, unless one or the other value is 0)
    if offset == 0: # skip the first one, which is a transition from a fake start value
        continue # resume loop with next item in lcp array
    elif lcp == lcp_array[offset - 1]:
        pass # same pattern (happens with repetition), so do nothing
    elif lcp > lcp_array[offset - 1]: # new prefix is longer than previous one, so start new pattern
        # can fill in end_position and frequency only when we encounter a shorter value in the LCP array
        # start_position is number of patterns that are the same 
        open_block_stack.append(Block(token_count = lcp, start_position = offset - 1, end_position = -1, all_start_positions = [], frequency = -1, how_created = 1))
    else: # new prefix is shorter than previous one, so:
            # 1. close open blocks with higher values
            # 2. do something else
        while open_block_stack and open_block_stack[-1].token_count > lcp: # if an open block is longer than the current length, pop and close it
            block_being_modified = open_block_stack.pop()
            block_being_modified.end_position = offset - 1
            block_being_modified.frequency = block_being_modified.end_position - block_being_modified.start_position + 1
            blocks.append(block_being_modified)
        if lcp == 0: # if 0, stop after clearing the stack
            continue
        # not 0, and: either 1) stack is empty, or 2) top has an lcp value equal to current, or 3) an lcp value less than current
        if not open_block_stack: # stack is empty, so hidden shorter block; create new block that starts at start position of last closed block
            open_block_stack.append(Block(token_count = lcp, start_position = blocks[-1].start_position, end_position = -1, all_start_positions = [], frequency = -1, how_created = 2))
        elif open_block_stack[-1].token_count == lcp: # stack has value same length as current, so do nothing
            pass
        else: # stack has value less than current, so extends shorter block; create new block, but where?
            # TODO: why does this work?
            open_block_stack.append(Block(token_count = lcp, start_position = blocks[-1].start_position, end_position = -1, all_start_positions = [], frequency = -1, how_created = 3))
            # print(open_block_stack)
            # print(blocks[-1])
        # if equal to current length, do nothing; it's open, but we can't close it yet
while open_block_stack: # pop anything left in open_block_stack
    block_being_modified = open_block_stack.pop()
    block_being_modified.end_position = len(lcp_array) - 1
    block_being_modified.frequency = block_being_modified.end_position - block_being_modified.start_position + 1
    blocks.append(block_being_modified)

for block in blocks: # diagnostic
    # suffix_array is offsets of start positions of suffixes (in alphabetical order) into token_array
    # block.start_position is offset of suffix into suffix_array, which is one less than offset into lcp_array
    # we slice token_array:
    #   start of slice is offset into token_array (by way of suffix_array by way of lcp_array) of first ngram token
    #   length of slice adds the length of the lcp interval (part of lcp array that represents pattern, which = length of ngram) to the start of the slice
    print(block)
    print(token_array[suffix_array[block.start_position]:suffix_array[block.start_position] + block.token_count])
if open_block_stack: # diagnostic; should be empty
    print('ERROR: open_block_stack should be empty')
    print(open_block_stack)

[(0, 0), (1, 0), (2, 2), (3, 0), (4, 1), (5, 0), (6, 1), (7, 0), (8, 1), (9, 0), (10, 2), (11, 1), (12, 2)]
Block(token_count=2, start_position=1, end_position=2, all_start_positions=[], frequency=2, how_created=1)
['and', 'the']
Block(token_count=1, start_position=3, end_position=4, all_start_positions=[], frequency=2, how_created=1)
['black']
Block(token_count=1, start_position=5, end_position=6, all_start_positions=[], frequency=2, how_created=1)
['cat']
Block(token_count=1, start_position=7, end_position=8, all_start_positions=[], frequency=2, how_created=1)
['red']
Block(token_count=2, start_position=9, end_position=10, all_start_positions=[], frequency=2, how_created=1)
['the', 'black']
Block(token_count=2, start_position=11, end_position=12, all_start_positions=[], frequency=2, how_created=1)
['the', 'red']
Block(token_count=1, start_position=9, end_position=12, all_start_positions=[], frequency=4, how_created=2)
['the']


### Taking stock

* Previously we had a list of ngrams, with locations in witnesses. This **new implementation gives us the ngrams in the patterns**.
* Previously repeated tokens were separate ngrams, while here they are combined in a block. The **blocks in the new version, then, don't record the locations of the ngrams in the witnesses**, and therefore also do not record repetition.
* Before we can produce an alignment, then **we need to get from the blocks to the locations in the witnesses**.

Each pattern is part of the suffix array, and the entries in the suffix array know which tokens in the witnesses correspond to the pattern. The blocks record start and end positions in the suffix array.

In [13]:
suffix_array

[6, 2, 9, 8, 4, 12, 5, 1, 11, 7, 3, 0, 10]

Block 1 starts at suffix_array position 1 and ends at 2. This means that tokens 2 (position 1) and 9 (position 2) are the (only) two start positions of the two instances of the bigram 'and the' in the token_array.

In [14]:
# If we enumerate the token array and look at tokens 2 and 9, we see the two instances of the bigram 'and the'.

[(index, value) for index,value in enumerate(token_array)]

[(0, 'the'),
 (1, 'red'),
 (2, 'and'),
 (3, 'the'),
 (4, 'black'),
 (5, 'cat'),
 (6, ' # '),
 (7, 'the'),
 (8, 'black'),
 (9, 'and'),
 (10, 'the'),
 (11, 'red'),
 (12, 'cat')]

If we iterate over the blocks, for each block we can get its start position(s) in the token array and the ngrams themselves (not in token-array order, but we can sort later). We can then sort them by token position to get a list of ngrams for both witnesses in token order (which we need because we do our alignment from left to right). The token array doesn't distinguish the witnesses explicitly, but if we subtract the length of w1 from a position, add 1, and get a positive number, we're in w2.

We call each occurrence of a pattern at a location in a witness an **instance**. We need to build a data structure that stores instances, which will store ngram, offset, and witness identifier (for convenience; we compute it from the offset into the token_array).

In [15]:
from typing import List
@dataclass
class Instance:
    instance_offset: int
    block_id: int # offset into list of blocks
    start_token: int # start position in token array
    ngram_length: int # length of ngram
    witness: int # 0 or 1

In [16]:
instances = []
for index, block in enumerate(blocks): # compute all start tokens just once for each block
    _all_start_tokens = suffix_array[block.start_position: block.end_position + 1]
    _all_start_tokens.sort()
    block.all_start_positions = _all_start_tokens # save it to the block
    for suffix_array_offset in range(block.start_position, block.end_position+1): # each will be a unique instance
        _instance_offset = -1
        _block_id = index
        _start_token = suffix_array[suffix_array_offset]
        _ngram_length = block.token_count
        _witness = 1 if _start_token > len(witnesses[0]) else 0
        instances.append(Instance(instance_offset = _instance_offset, block_id = _block_id, start_token = _start_token, ngram_length = _ngram_length, witness = _witness))
instances.sort(key = lambda x: x.start_token)
instance_by_instance_offset = {}
for index, instance in enumerate(instances):
    instance.instance_offset = index
    instance_by_instance_offset[index] = instance
pointer_w0 = 0
pointer_w1 = len(list(filter(lambda x: x.witness == 0, instances)))

In [17]:
blocks

[Block(token_count=2, start_position=1, end_position=2, all_start_positions=[2, 9], frequency=2, how_created=1),
 Block(token_count=1, start_position=3, end_position=4, all_start_positions=[4, 8], frequency=2, how_created=1),
 Block(token_count=1, start_position=5, end_position=6, all_start_positions=[5, 12], frequency=2, how_created=1),
 Block(token_count=1, start_position=7, end_position=8, all_start_positions=[1, 11], frequency=2, how_created=1),
 Block(token_count=2, start_position=9, end_position=10, all_start_positions=[3, 7], frequency=2, how_created=1),
 Block(token_count=2, start_position=11, end_position=12, all_start_positions=[0, 10], frequency=2, how_created=1),
 Block(token_count=1, start_position=9, end_position=12, all_start_positions=[0, 3, 7, 10], frequency=4, how_created=2)]

In [18]:
instances # take a look

[Instance(instance_offset=0, block_id=5, start_token=0, ngram_length=2, witness=0),
 Instance(instance_offset=1, block_id=6, start_token=0, ngram_length=1, witness=0),
 Instance(instance_offset=2, block_id=3, start_token=1, ngram_length=1, witness=0),
 Instance(instance_offset=3, block_id=0, start_token=2, ngram_length=2, witness=0),
 Instance(instance_offset=4, block_id=4, start_token=3, ngram_length=2, witness=0),
 Instance(instance_offset=5, block_id=6, start_token=3, ngram_length=1, witness=0),
 Instance(instance_offset=6, block_id=1, start_token=4, ngram_length=1, witness=0),
 Instance(instance_offset=7, block_id=2, start_token=5, ngram_length=1, witness=0),
 Instance(instance_offset=8, block_id=4, start_token=7, ngram_length=2, witness=1),
 Instance(instance_offset=9, block_id=6, start_token=7, ngram_length=1, witness=1),
 Instance(instance_offset=10, block_id=1, start_token=8, ngram_length=1, witness=1),
 Instance(instance_offset=11, block_id=0, start_token=9, ngram_length=2, wi

In [19]:
instance_by_instance_offset # take a look

{0: Instance(instance_offset=0, block_id=5, start_token=0, ngram_length=2, witness=0),
 1: Instance(instance_offset=1, block_id=6, start_token=0, ngram_length=1, witness=0),
 2: Instance(instance_offset=2, block_id=3, start_token=1, ngram_length=1, witness=0),
 3: Instance(instance_offset=3, block_id=0, start_token=2, ngram_length=2, witness=0),
 4: Instance(instance_offset=4, block_id=4, start_token=3, ngram_length=2, witness=0),
 5: Instance(instance_offset=5, block_id=6, start_token=3, ngram_length=1, witness=0),
 6: Instance(instance_offset=6, block_id=1, start_token=4, ngram_length=1, witness=0),
 7: Instance(instance_offset=7, block_id=2, start_token=5, ngram_length=1, witness=0),
 8: Instance(instance_offset=8, block_id=4, start_token=7, ngram_length=2, witness=1),
 9: Instance(instance_offset=9, block_id=6, start_token=7, ngram_length=1, witness=1),
 10: Instance(instance_offset=10, block_id=1, start_token=8, ngram_length=1, witness=1),
 11: Instance(instance_offset=11, block_i

In [20]:
w0_last_instance_offset = len(list(filter(lambda x: x.witness == 0, instances)))
w1_last_instance_offset = len(instances)
print(w0_last_instance_offset, w1_last_instance_offset) # take a look

8 16


In [21]:
# create mapping from blocks to instances
from collections import defaultdict
instances_by_block = defaultdict(list)
for index, instance in enumerate(instances):
    instances_by_block[instance.block_id].append(index)
instances_by_block # take a look

defaultdict(list,
            {5: [0, 12],
             6: [1, 5, 9, 13],
             3: [2, 14],
             0: [3, 11],
             4: [4, 8],
             1: [6, 10],
             2: [7, 15]})

### Create decision graph

Three options are first in w0, first in w1, and first in both

In [22]:
# create dataclasses for decision graph and its nodes and edges

from typing import Set

@dataclass(unsafe_hash=True)
class Decision_Graph_Node:
    id: int
    witness_0_instance_offset: int # we can get the start token from the instance
    witness_1_instance_offset: int
    ngram_length: int # same in both witness because same ngram ... doh!

@dataclass
class Decision_Graph_Edge:
    source: int # node id
    target: int # node id

@dataclass
class Decision_Graph:
    nodes: Set[Decision_Graph_Node]
    edges: Set[Decision_Graph_Edge]

In [23]:
# we need to know when we've seen all of the instances for a witness
w0_instance_end = len(list(filter(lambda x: x.witness == 0,instances))) # offset of last w0 instance in instances list
w1_instance_end = len(instances) # offset of last w1 instance in instances list
print(w0_instance_end, w1_instance_end)

8 16


In [24]:
# create root node in decision graph
# the instance offsets are one before the first real instance, so 
#   -1 to represent the position before w0
#   the last instance offset for w0 to represent the position before w1
# the ngram length for the root, 1, is a fake to make the stupid arithmetic work
decision_graph = Decision_Graph(nodes = set(), edges = set())
root_node = Decision_Graph_Node(id = 0, witness_0_instance_offset = -1, witness_1_instance_offset = w0_instance_end, ngram_length = 1)
decision_graph.nodes.add(root_node)

In [25]:
decision_graph.nodes # take a look

{Decision_Graph_Node(id=0, witness_0_instance_offset=-1, witness_1_instance_offset=8, ngram_length=1)}

In [26]:
# generator function to loop over w1 instances looking for match
def next_matching_w1_instance(w1_instance_offset):
    while w1_instance_offset < w1_last_instance_offset:
        yield w1_instance_offset
        w1_instance_offset -= 1

# create other nodes in decision graph
def add_children(parent: Decision_Graph_Node): # modifies global in place, does not return
    # create three nodes: next in w0, next in w1, next in both, but ...
    #   ... node may already exist
    # reset pointers to after current instance (i.e., don't always increment by 1)
    # recur if there are still instances to consider
    #
    ######
    # add next in w0 and recur if necessary
    ######
    #
    # find first next instance in w0 that has a matching instance for w1 that is after the parent instance in w1 offset
    #
    # for each position to the right in w0
    #   (inner loop)
    #   find next matching instance in w1 until end of w1 instances
    #   if w1 instance is to the right of w1 position of parent, create a node and break out of inner loop, returning break signal
    #   otherwise loop
    #   (end of inner loop)
    #   if break signal, break out of outer loop
    #   otherwise loop to check next position in w0

    for w0_offset in range(parent.witness_0_instance_offset, w0_last_instance_offset):
        print(w0_offset)
        w1_instances = next_matching_w1_instance(parent.witness_1_instance_offset)
        print(w1_instances.__next__())


#     next_instance_for_w0 = instances[parent.witness_0_instance_offset + 1:w0_instance_end][0]
#     matching_instance_in_w1 = instances[next(filter(lambda x: instances[x].witness == 1, instances_by_block[next_instance_for_w0.block_id]))]
#     if matching_instance_in_w1.instance_offset > parent.witness_1_instance_offset:
#         _new_node_id = len(decision_graph.nodes)
#         _new_node = Decision_Graph_Node(id = _new_node_id, witness_0_instance_offset = next_instance_for_w0.instance_offset, witness_1_instance_offset = matching_instance_in_w1.instance_offset, ngram_length = next_instance_for_w0.ngram_length)
#         decision_graph.nodes.add(_new_node)
#         # diagnostic
#         _current_ngram = witnesses[0][next_instance_for_w0.start_token:next_instance_for_w0.start_token + next_instance_for_w0.ngram_length]
#         print('w0: ', next_instance_for_w0)
#         print('w1: ', matching_instance_in_w1)
#         print(_new_node, _current_ngram)
#         # recur if there's more to do
#         if next_instance_for_w0.instance_offset < w0_last_instance_offset and matching_instance_in_w1.instance_offset < w1_last_instance_offset:
#             add_children(_new_node)

In [27]:
add_children(root_node)

-1
8
0
8
1
8
2
8
3
8
4
8
5
8
6
8
7
8


In [28]:
decision_graph.nodes

{Decision_Graph_Node(id=0, witness_0_instance_offset=-1, witness_1_instance_offset=8, ngram_length=1)}